In [3]:
import warnings
warnings.filterwarnings('ignore')

import re
import string
import os
from collections import defaultdict
from collections import Counter

# General science/numeric tools
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

from sklearn import metrics
from sklearn.metrics import confusion_matrix,accuracy_score,roc_auc_score,roc_curve,auc,f1_score

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

# NLTK for NLP tools such as stemming, lemmatizing
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('punkt')
nltk.download('wordnet')
stop = set(stopwords.words('english'))

from sklearn.feature_extraction.text import CountVectorizer

import gensim
from tqdm.notebook import tqdm



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [4]:
# from google.colab import drive
# drive.mount('/gdrive')
# cd /gdrive/My Drive

In [5]:
# to access my drive to read the csv-files
from google.colab import drive
drive.mount('/content/drive')

# link to the folder of Colab Notebooks 
%cd '/content/drive/MyDrive/Colab Notebooks'

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks


In [6]:
df = pd.read_csv('train_bodies.csv')
df1 = pd.read_csv('train_stances.csv')

In [7]:


# df = df.rename(columns={'Body ID': 'Body_ID'})
# df1 = df1.rename(columns={'Body ID': 'Body_ID'})

# df_ = df.reindex(range(len(df1)), fill_value=None)
df_merge = pd.merge(df, df1, on='Body ID')
df_merge.sample(10)
print(len(df_merge))

df.sample(10)

print(len(df))
print(len(df1))

49972
1683
49972


In [8]:
TEXT_C = ['articleBody','Headline']
TARGET_C = 'Stance'

df_merge['Stance'].replace(['agree', 'disagree', 'discuss'], 'related',inplace=True)

# df_merge.sample(20)
print(df_merge['Stance'].value_counts())

unrelated    36545
related      13427
Name: Stance, dtype: int64


In [9]:
# TEXT_C = ['articleBody','Headline']
# TARGET_C = 'Stance'

X_train, X_val,  y_train, y_val = train_test_split(df_merge[TEXT_C], 
                                                  df_merge[TARGET_C].values,
                                                  test_size=0.20,
                                                  random_state=42)
# check the size of our datasets
print('Size of training set:',X_train.shape)
print('Size of validation set:',X_val.shape)

Size of training set: (39977, 2)
Size of validation set: (9995, 2)


In [10]:
X_train

,articleBody,Headline
45131,It's bad enough how much they crop it. Now I n...,WHO says reports of suspected Ebola cases in I...
48011,ISIS militants released gruesome video footage...,Islamic Militants Post Video Claiming to Show ...
26530,I aborted my baby because it was a boy,"Batmobile Stolen From ""Batman v Superman: Dawn..."
30200,(CNN) -- Boko Haram laughed off Nigeria's anno...,Pictured: Libyan Islamist rebels pose with pla...
7195,TEXAS CITY — A local plumber is being flooded ...,Islamist terrorists Jabhat Ansar al-Deen tweet...
...,...,...
11284,(CNN) -- The mystery surrounding North Korea's...,HBO streaming service could launch in April fo...
44732,You know that tendency we have to judge people...,NEWS/ You'll Never Guess How a Homeless Man Sp...
38158,When a report went viral that NBC meteorologis...,"Hostage David Haines' Murder 'Evil', PM Says"
860,"New Delhi: AK Verma, an executive engineer at ...",ISIS leader dead?


In [11]:
# from sklearn.model_selection import train_test_split

# X_test, X_val, Y_test, Y_val = train_test_split(X_val, y_val, test_size=0.5, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=42)


# X_train_tf = X_train['articleBody'] + X_train['Headline']
# X_val_tf= X_val['articleBody'] + X_val['Headline']
# X_test_tf = X_test['articleBody'] + X_test['Headline']

# turn y-train and y_test into dataframe format to use the replace function
# y_train_tf = pd.DataFrame(y_train)
# y_val_tf = pd.DataFrame(y_val)
# y_test_tf = pd.DataFrame(y_test)

# y_test = np.where(y_test == 'unrelated', 0, 1)

# y_test = np.where(arr == 'related', 1, 0)
# turn 'unrelated' to 0, turn 'related' to 1, it is easier to comnbine by using tokens.
# y_train_tf = y_train_tf.replace({'unrelated': 0, 'related': 1})
# y_train_tf = y_val_tf.replace({'unrelated': 0, 'related': 1})
# y_test_tf = y_test_tf.replace({'unrelated': 0, 'related': 1})

In [12]:
X_train_tf = X_train['articleBody'] + X_train['Headline']
X_val_tf= X_val['articleBody'] + X_val['Headline']
X_test_tf = X_test['articleBody'] + X_test['Headline']

y_train_1 = []

for e in y_train:
  if e == 'unrelated':
    y_train_1.append(0)
  else:
    y_train_1.append(1)
print(y_train_1)


y_val_1 = []

for e in y_val:
  if e == 'unrelated':
    y_val_1.append(0)
  else:
    y_val_1.append(1)
print(y_val_1)

y_test_1 = []

for e in y_test:
  if e == 'unrelated':
    y_test_1.append(0)
  else:
    y_test_1.append(1)
print(y_test_1)

[0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 

In [13]:
y_train

array(['unrelated', 'related', 'unrelated', ..., 'unrelated', 'unrelated',
       'unrelated'], dtype=object)

fix the above prob

In [14]:
# X_train = X_train['Headline'] + X_train['articleBody']
# X_val = X_val['Headline'] + X_val['articleBody']

## it doesn't work because it had changed to a different dataframe (this code has been used once, it would change)

combine two columns to one column, to apply the two codes following

train = tfidf.fit_transform(X_train.values.astype('U'))

val = tfidf.transform(X_val.values.astype('U'))

In [15]:
# df = df_merge1.applymap(lambda x: str(x).lower() if isinstance(x, str) else ' '.join(map(str, x)).lower())
# train


In [16]:
X_train

,articleBody,Headline
45131,It's bad enough how much they crop it. Now I n...,WHO says reports of suspected Ebola cases in I...
48011,ISIS militants released gruesome video footage...,Islamic Militants Post Video Claiming to Show ...
26530,I aborted my baby because it was a boy,"Batmobile Stolen From ""Batman v Superman: Dawn..."
30200,(CNN) -- Boko Haram laughed off Nigeria's anno...,Pictured: Libyan Islamist rebels pose with pla...
7195,TEXAS CITY — A local plumber is being flooded ...,Islamist terrorists Jabhat Ansar al-Deen tweet...
...,...,...
11284,(CNN) -- The mystery surrounding North Korea's...,HBO streaming service could launch in April fo...
44732,You know that tendency we have to judge people...,NEWS/ You'll Never Guess How a Homeless Man Sp...
38158,When a report went viral that NBC meteorologis...,"Hostage David Haines' Murder 'Evil', PM Says"
860,"New Delhi: AK Verma, an executive engineer at ...",ISIS leader dead?


In [17]:
# my_array = np.array(X_train)
# my_array_str = my_array.astype(str)

# # apply the lower() method to the string
# my_array_str_lower = np.char.lower(my_array_str)

# # create a TfidfVectorizer instance and fit the lowercased strings
# tfidf = TfidfVectorizer()
# tfidf.fit(my_array_str_lower)

In [18]:
# Try multiple ways of calculating features.
#   Create the numericalizer TFIDF for lowercase
tfidf = TfidfVectorizer(decode_error='ignore', lowercase=True, min_df=2)
#   Numericalize the train dataset
train_tfidf = tfidf.fit_transform(X_train_tf.values.astype('U'))
#   Numericalize the validation dataset
val_tfidf = tfidf.transform(X_val_tf.values.astype('U'))
#   Numericalize the test dataset
test_tfidf = tfidf.transform(X_test_tf.values.astype('U'))

In [22]:
print('Train size: ',train_tfidf.shape)
print('Val size: ',val_tfidf.shape)
print('Test size: ',test_tfidf.shape)
train_tfidf.shape[0]

Train size:  (39977, 23733)
Val size:  (4997, 23733)
Test size:  (4998, 23733)


39977

In [23]:
# from sklearn.feature_extraction.text import CountVectorizer


# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(train)
# vectorizer.get_feature_names_out()

In [24]:
# ## extract vocab from tfidf
# vocab = tfidf.get_feature_names_out()
# len(vocab)

In [25]:
len(X_val)

4997

In [26]:
len(X_test)

4998

In [27]:
# # turn 'unrelated' to 0, turn 'related' to 1, it is easier to comnbine by using tokens.
# y_train_bert = y_train_bert.replace({'unrelated': 0, 'related': 1})

In [28]:
# label_map = {'unrelated': 0, 'related': 1}

# # use np.where to map the class labels to new values
# y_test_mapped = np.where(y_test=='unrelated', label_map['unrelated'], label_map['related'])

# y_test=pd.DataFrame(y_test)
# y_test = y_test.replace({'unrelated': 0, 'related': 1})
# y_pred = model.predict(test)

# plot_roc_curve(y_test_1, y_pred)


Bert 

In [29]:
# !pip install torchtext==0.14.1
# !pip install torchdata==0.5.1
!pip install transformers
# !pip install portalocker
# !pip install torchtext.legacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.2 MB/s eta 0:00:00


In [30]:
y_train

array(['unrelated', 'related', 'unrelated', ..., 'unrelated', 'unrelated',
       'unrelated'], dtype=object)

In [31]:
import torch
import random
from torchtext import datasets
from torchtext.data.functional import to_map_style_dataset

y_train_2 = []

for e in y_train:
  if e == 'unrelated':
    y_train_2.append(0)
  else:
    y_train_2.append(1)
print(y_train_2)

y_val_2 = []

for e in y_val:
  if e == 'unrelated':
    y_val_2.append(0)
  else:
    y_val_2.append(1)
print(y_val_2)

y_test_2 = []

for e in y_test:
  if e == 'unrelated':
    y_test_2.append(0)
  else:
    y_test_2.append(1)
print(y_test_2)


from torch.utils.data import ConcatDataset

# re-integrate X_train and y_train (y_train_2 is the version which shows 0 and 1 instead of unrelated and related)
X_train_df = X_train
X_train_df['Stance'] = y_train_2

X_val_df = X_val
X_val_df['Stance'] = y_val_2

X_test_df = X_test
X_test_df['Stance'] = y_test_2

# train_dataset_c = ConcatDataset([[X_train], [y_train_dataset]])
# val_dataset_c = ConcatDataset([X_val, y_val_dataset])
# test_dataset_c = ConcatDataset([X_test, y_test_dataset])

# print(train_dataset_c[0]['Headline'][0])
# print(train_dataset_c[0]['articleBody'][0])


X_train_dataset  = to_map_style_dataset(X_train_df.values)
# y_train_dataset = to_map_style_dataset(y_train_2)
X_val_dataset  = to_map_style_dataset(X_val_df.values)
# y_val_dataset = to_map_style_dataset(y_val_2)
X_test_dataset   = to_map_style_dataset(X_test_df.values)
# y_test_dataset = to_map_style_dataset(y_test_2)


X_train_dataset = list(X_train_dataset)
# y_train_dataset = list(y_train_dataset)
X_val_dataset = list(X_val_dataset)
# y_val_dataset = list(y_val_dataset)
X_test_dataset = list(X_test_dataset)
# y_test_dataset = list(y_test_dataset)
# Split train into training set and validation set
# print(y_test_dataset[0])


[0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 

In [32]:
# !pip install torchtext==0.14.1
# !pip install torchdata==0.5.1

# X_val
# y_val_dataset

In [33]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [34]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token, eos_token, pad_token, unk_token)


[CLS] [SEP] [PAD] [UNK]


In [35]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [36]:
X_train_bert = pd.DataFrame(X_train['articleBody'])
X_train_bert = pd.DataFrame({'articleBody': X_train['articleBody'], 'Headline': X_train['Headline']})
y_train_bert = pd.DataFrame(y_train)
# turn 'unrelated' to 0, turn 'related' to 1, it is easier to comnbine by using tokens.
y_train_bert = y_train_bert.replace({'unrelated': 0, 'related': 1})
# X_val= X_val['articleBody'] + X_val['Headline']
# X_test= X_test['articleBody'] + X_test['Headline']
X_train_bert

,articleBody,Headline
45131,It's bad enough how much they crop it. Now I n...,WHO says reports of suspected Ebola cases in I...
48011,ISIS militants released gruesome video footage...,Islamic Militants Post Video Claiming to Show ...
26530,I aborted my baby because it was a boy,"Batmobile Stolen From ""Batman v Superman: Dawn..."
30200,(CNN) -- Boko Haram laughed off Nigeria's anno...,Pictured: Libyan Islamist rebels pose with pla...
7195,TEXAS CITY — A local plumber is being flooded ...,Islamist terrorists Jabhat Ansar al-Deen tweet...
...,...,...
11284,(CNN) -- The mystery surrounding North Korea's...,HBO streaming service could launch in April fo...
44732,You know that tendency we have to judge people...,NEWS/ You'll Never Guess How a Homeless Man Sp...
38158,When a report went viral that NBC meteorologis...,"Hostage David Haines' Murder 'Evil', PM Says"
860,"New Delhi: AK Verma, an executive engineer at ...",ISIS leader dead?


In [37]:
# combined_df = pd.DataFrame(X_train_bert.apply(lambda x: x['articleBody'] + ' ' + x['Headline'], axis=1), columns=['combined'])
# combined_df

In [38]:
# y_train_bert = y_train_bert.replace({'unrelated': 0, 'related': 1})
# y_train_bert

ba unrelated and related biancheng o and 1, shi jiqi nenggou shibie, ranhou zai yon for loop jiang articleBody he headline ping dao yiqi

In [39]:
# linked_tokens = []

# for i in range(len(df)):
#     # get the tokens in the current row
#     token1 = df.loc[i, 'token1']
#     token2 = df.loc[i, 'token2']
#     # link the tokens with an underscore and append to the list
#     linked_tokens.append(token1 + '_' + token2)


In [40]:
# from torch.utils.data import ConcatDataset

# X_train_df = X_train
# # result = pd.concat([X_train, df2], axis=1)

# X_train_df['Stance'] = y_train_dataset

# train_dataset_c = ConcatDataset([[X_train], [y_train_dataset]])
# val_dataset_c = ConcatDataset([X_val, y_val_dataset])
# test_dataset_c = ConcatDataset([X_test, y_test_dataset])

# # print(train_dataset_c[0]['Headline'][0])
# # print(train_dataset_c[0]['articleBody'][0])


# # print(train_dataset_c[1][0])
# # print(train_dataset_c[1][0]
# # train_dataset_c = list(train_dataset_c)
# # train_dataset_c[0]['Headline'][0]
# type(y_train_dataset)
# X_train_df
# X_train_dataset[0][0]

In [41]:
from torch.utils.data import DataLoader,Sampler
from torch.nn.utils.rnn import pad_sequence

BATCH_SIZE = 32


# integrate headline and articlebody in this step
def vectorize_batch(batch):
    '''Take a batch of (text,label) pairs and return tensors ready for input to the model.'''
    label_list, text_list = [], []
    # train_dataset_c[0]['Headline'][0]
    for (headline, body,stance) in batch:
      print(headline)
      print(body)
      print(stance)
      new_column_values = []
      # Use a for loop to iterate over each row in the dataframe
      # for i in range (len(X_train_bert)):
        # Add the new string to the new_column_values list
      label_list.append(int(stance)-1)
      tokens = tokenizer.convert_tokens_to_ids(tokenize_and_cut(headline))
      tokens1 = tokenizer.convert_tokens_to_ids(tokenize_and_cut(body))
      text_list.append(torch.tensor([init_token_idx] + tokens + [eos_token_idx]+tokens1))
      print(text_list)
    return pad_sequence(text_list,
                        padding_value=pad_token_idx,
                        batch_first=True), torch.tensor(label_list)
                                      
class BucketSampler(Sampler):
    def __init__(self, dataset, batch_size):
        # train_list = list(dataset)
        indices = [(i, len(tokenizer(s[0]))) for i, s in enumerate(dataset)]
        random.shuffle(indices)
        self.batch_size = batch_size
        
        # create pool of indices with similar lengths 
        self.pooled_indices = []
        for i in range(0, len(indices), self.batch_size * 100):
            self.pooled_indices.extend(sorted(indices[i:i + self.batch_size * 100], key=lambda x: x[1]))
        self.pooled_indices = [x[0] for x in self.pooled_indices]
        
    def __iter__(self):
        self.count = 0
        return self
    
    def __next__(self):
        # yield indices for current batch
        if self.count >= len(self.pooled_indices)-self.batch_size:
            raise StopIteration
        self.count += 1
        
        for i in range(0, len(self.pooled_indices), self.batch_size):
            return self.pooled_indices[self.count:self.count + self.batch_size]
        
    def __len__(self):
        return len(self.pooled_indices)


train_iterator  = DataLoader(X_train_dataset, collate_fn=vectorize_batch, batch_size=BATCH_SIZE)
valid_iterator  = DataLoader(X_val_dataset, collate_fn=vectorize_batch, batch_size=BATCH_SIZE)
test_iterator   = DataLoader(X_test_dataset,  collate_fn=vectorize_batch, batch_size=BATCH_SIZE)

In [42]:
# train_dataset
# train_dataset
# for i, s in enumerate(list(X_train.values)):
  # print(s[1])
# list(X_train.values)


# list(train_dataset[1])

# print(train_dataset.)
# print(train_dataset.labels)

In [43]:
# import itertools

# for batch in train_iterator:
#     X, Y = itertools.islice(batch, 2)  # limit to two values
#     print("input_ids:", X)
#     print("labels:", Y)
#     break
# vectorize_batch(zip(X_train_dataset, y_train_dataset))
# X_train_dataset
# vectorize_batch(X_train_dataset)

In [44]:
for X_train, Y_train in train_iterator:
    print("input_ids:",X_train)
    print("labels:", Y_train)
    break

Streaming output truncated to the last 5000 lines.
        27914,  6290,  1010,  2040,  2038,  3079,  1998,  3498,  2928,  1011,
         1015, 27902,  1999,  3146,  2103,  2005,  1996,  2627,  3590,  2086,
         1010,  4484,  2009,  2001,  2010, 15373,  4744,  1999,  1996,  3861,
         1012,  2002,  2056,  2002,  2053,  2936,  3079,  1996,  4316,  1998,
         2018,  2053,  2801,  2129,  2009,  3092,  2039,  1999,  7795,  1012,
        15578, 25032, 27914,  6290,  2056,  2002,  7007,  1999,  1996,  4744,
         2000,  2019,  8285,  9323, 16743,  5605,  2093,  2086,  3283,  1012,
         2002,  2788,  3138,  1996, 11703,  9777,  2125,  2010,  4683,  2043,
         2002, 15187,  2068,  2021,  2002,  2187,  2009,  2006,  2023,  4744,
         2007,  1996, 17626,  2008,  8285,  9323,  2052,  6366,  2009,  1012,
         1523,  2027,  2020,  4011,  2000,  2031,  2589,  2009,  1998,  2009,
         3504,  2066,  2027,  2134,  1521,  1056,  2079,  2009,  1010,  1524,
        15578

In [45]:
for X_val, Y_val in valid_iterator:
    print("input_ids:",X_val)
    print("labels:", Y_val)
    break

Streaming output truncated to the last 5000 lines.
         2331,  1997,  1996,  2034,  2137,  1999,  5712,  2144,  1996,  2149,
         2510,  6780,  1999,  2249,  1010,  3778,  2003, 15986,  2006,  8112,
         2000,  7818,  2010,  2525,  3652,  1998, 16095,  8458,  3560,  2250,
         2162,  2114, 18301,  1012,  1996,  3090,  3800,  1997,  3053,  2048,
         3134,  1997,  8647,  2038,  2333,  2013,  1996,  5343,  1997,  3262,
         8038,  5831,  4305,  8956,  9272,  2012,  3891,  1997, 14052,  2000,
         4346,  2250,  3104,  2005, 15553,  1998,  8956,  2749, 23277,  4355,
         2075,  1996, 23972,  8995,  9587, 23722,  5477,  2185,  2013, 18301,
         1012, 14895,  2015,  2000,  8112,  4197,  2041,  2013,  1996,  2707,
         2008,  8767,  2000,  4187,  6502,  2052,  3497, 25732,  1037,  2149,
        16360,  6935,  2389,  1012, 28305,  2098,  2090,  4455,  2000,  6033,
        18301,  1998,  6256,  1997,  2037,  9564, 11581,  2063,  1010,  8112,
         1998

In [46]:
for X_test, Y_test in test_iterator:
    print("input_ids:",X_test)
    print("labels:", Y_test)
    break

Streaming output truncated to the last 5000 lines.
         9919,  1520,  9458,  4702,  1521,  2081,  1002,  5824,  2454,  2006,
         4518,  3006]), tensor([  101,  1996,  2866,  4484,  1996,  2678,  4760,  1996,  2022,  4974,
         2075,  1997,  4681,  7309,  2585, 15030,  5267,  1012,   102,  1005,
         2045,  2003,  2053,  5787,  2000,  2037,  2139, 18098, 18891,  3723,
         1005,  5499,  2110,  5438,  2388,  2014,  2219,  2365]), tensor([  101,  2343,  8112,  2056,  9317,  2008,  1996,  2142,  2163,  1523,
         2097,  2022,  6819, 20142,  4630,  1998,  2057,  2097,  2022, 21660,
         1524,  2114,  5499,  2110, 17671,  1998,  2052,  1523,  2079,  2054,
         1521,  1055,  4072,  2000,  2156,  2008,  3425,  2003,  2589,  1524,
         2206,  1996,  2678,  2696,  5669,  7781,  1997,  2019,  2137,  4988,
         1012,  1523,  2651,  1996,  2972,  2088,  2003, 29279,  2012,  1996,
        12077,  4028,  1997,  2508, 17106,  2011,  1996,  9452,  2177, 19600,
 

Preprocess

try to use the same proprocess in bert-model and tf-idf model, because the overfiting could be solved by this, with the detial guidance in GPT

In [48]:
# train

In [53]:
import re
from nltk.corpus import stopwords
from transformers import BertTokenizer
import numpy as np
from scipy.sparse import csr_matrix
# Define a preprocessing function
def preprocess(text):
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize using Bert tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    tokens = tokenizer.tokenize(text)
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.lower() not in stop_words]
    # Convert tokens back to text
    text = ' '.join(tokens)
    return text

# Preprocess the data
# train_data = train_data.apply(preprocess)
# test_data = test_data.apply(preprocess)
train_tfidf1 = train_tfidf.toarray()
# train_tfidf2 = train_tfidf1.apply(preprocess)

for i in range(len(train_tfidf1)):
    train_tfidf1[i][-1] = preprocess(train_tfidf1[i][-1])

# Convert the dense matrix back to a sparse matrix
train_tfidf = scipy.sparse.csr_matrix(train_tfidf1)
# y_train_1 = preprocess(y_train_1.values)
## what format

AttributeError: ignored

In [54]:
train_tfidf

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.02663395, 0.        , ..., 0.        , 0.        ,
        0.        ]])

Build the Naive Bayes model on TF-IDF and Bert

In [ ]:
#Import the liberary
# import matplotlib.pyplot as plt

import seaborn as sn
%matplotlib inline
from sklearn import metrics
from sklearn.metrics import confusion_matrix,accuracy_score,roc_auc_score,roc_curve,auc,f1_score

In [ ]:
## TF-IDF
# create the model, train it on the train dataset and print the scores
model = MultinomialNB() # as implemented in sklearn
model.fit(train, y_train_1)
print("Train score:", model.score(train, y_train_1))
print("Validation score:", model.score(val, y_val_1))


In [ ]:
# X_numpy = X.numpy()  # convert X tensor to a NumPy array
# Y_numpy = Y.numpy()  # convert Y tensor to a NumPy array
# X

In [ ]:
# Y_test_numpy

In [ ]:
# Create the confusion matrix
def plot_confusion_matrix(y_test, y_pred):
    ''' Plot the confusion matrix for the target labels and predictions '''
    cm = confusion_matrix(y_test, y_pred)

    # Create a dataframe with the confusion matrix values
    df_cm = pd.DataFrame(cm, range(cm.shape[0]),
                  range(cm.shape[1]))

    # Plot the confusion matrix
    sn.set(font_scale=1.4) # for label size
    sn.heatmap(df_cm, annot=True,fmt='.0f',cmap="YlGnBu",annot_kws={"size": 10}) # font size
    plt.show()


    # ROC Curve
# plot no skill
# Calculate the points in the ROC curve


def plot_roc_curve(y_test, y_pred):
    ''' Plot the ROC curve for the target labels and predictions'''
    fpr, tpr, thresholds = roc_curve(y_test, y_pred, pos_label=1)
    roc_auc= auc(fpr,tpr)
    plt.figure(figsize=(12, 12))
    ax = plt.subplot(121)
    ax.set_aspect(1)
    
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [ ]:
y_pred = model.predict(test)
#print the classification report to highlight the accuracy with f1-score, precision and recall

print(metrics.classification_report(y_test_1, y_pred))
plot_confusion_matrix(y_test_1, y_pred)
plot_roc_curve(y_test_1, y_pred)
# label_map = {'unrelated': 0, 'related': 1}


In [ ]:
# ## Bert
# # create the model, train it on the train dataset and print the scores
# model = MultinomialNB() # as implemented in sklearn
# model.fit(X_numpy, Y_numpy)
# print("Train score:", model.score(train, y_train_1))
# print("Validation score:", model.score(val, y_val_1))

In [ ]:
## Bert

X_train_numpy = X_train.numpy()  # convert X tensor to a NumPy array
Y_train_numpy = Y_train.numpy()  # convert Y tensor to a NumPy array
X_val_numpy = X_val.numpy()  # convert X tensor to a NumPy array
Y_val_numpy = Y_val.numpy()  # convert Y tensor to a NumPy array
X_test_numpy = X_test.numpy()  # convert X tensor to a NumPy array
Y_test_numpy = Y_test.numpy()  # convert Y tensor to a NumPy array
# test_bert = np.concatenate((X_test_numpy, Y_test_numpy.reshape(-1, 1)), axis=1)


# create the model, train it on the train dataset and print the scores
model = MultinomialNB() # as implemented in sklearn
model.fit(X_train_numpy, Y_train_numpy)
print("Train score:", model.score(X_train_numpy, Y_train_numpy))
print("Validation score:", model.score(X_train_numpy, Y_train_numpy))

In [ ]:
model = MultinomialNB() # as implemented in sklearn
model.fit(X_test_numpy, Y_test_numpy)
print("Train score:", model.score(X_test_numpy, Y_test_numpy))
print("Validation score:", model.score(X_test_numpy, Y_test_numpy))

In [ ]:
# assuming X_train_numpy is the training data and X_test_numpy is the test data
print("Training data shape:", X_train_numpy.shape)
print("Test data shape:", X_test_numpy.shape)

# select the relevant features from the test data
X_test_selected = X_test_numpy[:, :X_train_numpy.shape[1]]

# check that the shapes match
print("Selected test data shape:", X_test_selected.shape)

# make predictions on the modified test data
y_pred = model.predict(X_test_selected)

In [ ]:
# test
# print("Training data shape:", X_train_numpy.shape)  # replace with the actual variable name for the training features
# print("Test data shape:", test_bert.shape)

In [ ]:
# anticipate the result from X_test
# y_pred = model.predict(X_test_numpy)
#print the classification report to highlight the accuracy with f1-score, precision and recall

print(metrics.classification_report(Y_test_numpy, y_pred))
plot_confusion_matrix(Y_test_numpy, y_pred)
plot_roc_curve(Y_test_numpy, y_pred)
# label_map = {'unrelated': 0, 'related': 1}

In [ ]:
import matplotlib.pyplot as plt
import torch
from torchtext import data
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import spacy
from torchdata.datapipes.iter import FileOpener, IterableWrapper, CSVParser
import os

In [ ]:
# device = torch.device('cuda:0' if torch.cuda.is_available()
#                        else 'cpu')

In [ ]:
# def TweetClassificationDataset(root, split, shuffle=False):
#     '''Dataset loading function'''
#     filename = os.path.join(root, "dataset_" + split + ".csv")
#     data_pipe = FileOpener([filename], encoding="utf-8") # open the file
#     data_pipe = CSVParser(data_pipe, skip_lines=1)       # read it as a CSV file
#     data_pipe = data_pipe.drop([0])                      # delete columns we're not using
#     return data_pipe.shuffle().set_shuffle(shuffle)


# print(list(TweetClassificationDataset("cache", split='train'))[:5])

In [ ]:
# from torchtext.data import get_tokenizer
# from torchtext.vocab import build_vocab_from_iterator
# import spacy
# from spacy.tokenizer import Tokenizer

# train_dataset  = TweetClassificationDataset("cache", split='train', shuffle=True)
# tokenizer      = get_tokenizer("spacy")

# def build_vocabulary(datasets):
#     for dataset in datasets:
#         for _, text  in dataset:
#             yield tokenizer(text.lower())

# vocab = build_vocab_from_iterator(build_vocabulary([train_dataset]), min_freq=3, specials=["<pad>", "<unk>"])
# vocab.set_default_index(vocab["<unk>"]) # if a word is not in the vocab - use the <unk> token

In [ ]:
vocab = tfidf.get_feature_names_out()
len(vocab)

Buile the LSTM model for TF-IDF

In [ ]:
from torch import nn
from torch.nn import functional as F

class TEXT_LSTM(nn.Module):
    def __init__(self, dimension=64):
        super(TEXT_LSTM, self).__init__()
        self.embedding_layer = nn.Embedding(len(vocab), 300)
        self.rnn = nn.LSTM(
            input_size=300,
            hidden_size=dimension,
            num_layers=1,
            bidirectional=True,
            batch_first=True)
        self.drop = nn.Dropout(p=0.5)
        self.fc = nn.Linear(2*dimension, 1)

    def forward(self, X_batch):
        embeddings = self.embedding_layer(X_batch)
        output, hidden = self.rnn(embeddings)
        text_fea = self.drop(output[:,-1])
        text_fea = self.fc(text_fea)
        text_fea = torch.squeeze(text_fea,1)
        text_out = torch.sigmoid(text_fea)
        return text_out

    
# Print out the layers
model = TEXT_LSTM()
print("Layers:")
for layer in model.children():
    print("\t{}".format(layer))

Training

In [ ]:
def save_checkpoint(save_path,model,optimizer,valid_loss):
    if save_path == None:
        return
    state_dict ={'model_state_dict': model.state_dict(),
                 'optimizer_state_dict': optimizer.state_dict(),
                 'valid_loss': valid_loss}
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')
    
def load_checkpoint(load_path,model,optimizer):
    if load_path == None:
        return
    state_dict = torch.load(load_path)
    print(f'Model loaded from <== {load_path}')
    
    model.load_state_dict(state_dict['model_state_dict'])
    optimizer.load_state_dict(state_dict['optimizer_state_dict'])
    
    return state_dict['valid_loss']

def save_metrics(save_path,train_loss_list, valid_loss_list,global_steps_list):
    if save_path==None:
        return
    state_dict= {'train_loss_list': train_loss_list,
                 'valid_loss_list': valid_loss_list,
                 'global_steps_list': global_steps_list}
    torch.save(state_dict, save_path)
    print(f'Metrics saved to ==> {save_path}')
    
def load_metrics(load_path):
    if load_path == None:
        return 
    state_dict = torch.load(load_path)
    print(f'Metrics loaded from <== {load_path}')
    return state_dict['train_loss_list'],state_dict['valid_loss_list'],state_dict['global_steps_list']

Build the main training loop

In [ ]:
train_loader = zip(train_tfidf,y_train_1)
val_loader = zip(val_tfidf,y_val_1)
test_loader = zip(test_tfidf, y_test_1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_tfidf.shape[0] 

In [ ]:
import torch
from scipy.sparse import csr_matrix
import scipy.sparse as sp
# # Assuming that `csr` is the `csr_matrix` object and `i` is the `int` object
# text = torch.tensor((text))  # convert csr_matrix to dense tensor
# labels = torch.tensor(labels)  # convert int to scalar tensor

# csr = sp.csr_matrix()

In [ ]:
def train(model,
          optimizer,
          criterion = nn.BCELoss(),
          train_loader = train_loader,
          valid_loader = val_loader,
          num_epochs = 5,
          eval_every = train_tfidf.shape[0] // 2,
          # file_path = "output",
          best_valid_loss = float("Inf")):
    
    # Initial running values
    running_loss = 0.0
    valid_running_loss = 0.0
    global_step = 0
    train_loss_list=[]
    valid_loss_list = []
    global_steps_list = []
    
    # Training Loop
    model.train()
    for epoch in range(num_epochs):
        losses = []
        for text, labels in train_loader:
            # Move everything to the right device
            text = text.toarray()
            text = torch.tensor(text)
            labels = torch.tensor(labels)
            text   = text.to(device)
            labels = labels.to(device)
            
            # Forward pass
            output = model(text.long())
            print(output.size())
            print(labels.size())
            # Calculate loss
            loss = criterion(output.squeeze(), labels.float().view(-1,1).squeeze())
            losses.append(loss.item())

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Update running values
            running_loss += loss.item()
            global_step  += 1
            
            
            # Evaluation step
            if global_step % eval_every == 0:
                model.eval()
                with torch.no_grad():
                    # Validation loop
                    for text, labels in valid_loader:
                        labels = labels.to(device)
                        text = text.to(device)
                        output = model(text)
                        
                        loss = criterion(output.to(torch.float32), labels.to(torch.float32))
                        valid_running_loss += loss.item()
                    
                # Evaluation
                average_train_loss = running_loss/eval_every
                average_valid_loss = valid_running_loss/len(valid_loader)
                train_loss_list.append(average_train_loss)
                valid_loss_list.append(average_valid_loss)
                global_steps_list.append(global_step)

                # Resetting running values
                running_loss = 0.0
                valid_running_loss = 0.0
                model.train()

                print('Epoch[{}/{}], Step [{}/{}], Train loss:{:.4f}, Valid loss:{:.4f}'.format(
                epoch+1, num_epochs, global_step, num_epochs*len(train_loader),average_train_loss,
                    average_valid_loss))

                # Checkpoint
                if best_valid_loss > average_valid_loss:
                    best_valid_loss = average_valid_loss
                    save_checkpoint(file_path+'/model.pt',model, optimizer, best_valid_loss)
                    save_metrics(file_path+'/metrics.pt',train_loss_list,valid_loss_list,global_steps_list)
                    
    save_metrics('/metrics.pt', train_loss_list,valid_loss_list,global_steps_list)
    print('Training is Done!')
    

loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
model = model.to(device)

train(model, optimizer, num_epochs=5)

In [ ]:
train.shape[0]